In [1]:
# https://adrien.barbaresi.eu/blog/trafilatura-main-text-content-python.html

import requests
import bs4
from bs4 import BeautifulSoup
import os


# Get archives

In [2]:
page_url = "https://kghosh.substack.com/archive"

page_sourced = requests.get(page_url).content 

html_content = BeautifulSoup(page_sourced, "html.parser")
# #main > div.archive-page.typography.use-theme-bg > div > div > div.portable-archive-list > div:nth-child(19) > div.post-preview-content > a.post-preview-title.newsletter
links = html_content.findAll('a', class_="post-preview-title newsletter")
print(len(links),links) 
pages = [i.get('href') for i in links if (not (i.get('href') is None))]

pages = [ i for i in pages  if "https://kghosh.substack.com/p/" in i ]
pages = list(set([ i for i in pages  if not i.endswith("comments") ]))
len(pages), pages

12 [<a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20230423">20230423</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20230416">20230416</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/fully-homomorphic-encryption">Fully Homomorphic Encryption</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20230408">20230408</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20230401">20230401</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/when-the-tide-of-ai-generated-texts">When the Tide of AI-generated Texts Rises</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20230325">20230325</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20230319">20230319</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20230312">2

(12,
 ['https://kghosh.substack.com/p/fully-homomorphic-encryption',
  'https://kghosh.substack.com/p/20230423',
  'https://kghosh.substack.com/p/20230408',
  'https://kghosh.substack.com/p/20230401',
  'https://kghosh.substack.com/p/20230319',
  'https://kghosh.substack.com/p/20230325',
  'https://kghosh.substack.com/p/20230305',
  'https://kghosh.substack.com/p/20230416',
  'https://kghosh.substack.com/p/20220225',
  'https://kghosh.substack.com/p/advisory-services-futures',
  'https://kghosh.substack.com/p/when-the-tide-of-ai-generated-texts',
  'https://kghosh.substack.com/p/20230312'])

In [3]:
for page in pages:
    name = page.split("/")[-1]

    if not os.path.exists(".cache/"+name):
        page_sourced = requests.get(page).content 
        html_content = BeautifulSoup(page_sourced, "html.parser")
        content = html_content.findAll('div', class_="body markup")

        with open(".cache/"+name, 'w') as f:
            f.write(str(content))
        print(name,"saved")
    else:
        print(name,"exists.")

fully-homomorphic-encryption exists.
20230423 saved
20230408 exists.
20230401 exists.
20230319 exists.
20230325 exists.
20230305 exists.
20230416 saved
20220225 exists.
advisory-services-futures exists.
when-the-tide-of-ai-generated-texts exists.
20230312 exists.


In [4]:
import glob
import pandas as pd


cached_pages = glob.glob(".cache/*")


In [10]:
print("There are",len(cached_pages),'pages saved.')
cached_pages.sort()
cached_pages[:3]

There are 42 pages saved.


['.cache/20220118', '.cache/20220128', '.cache/20220212']

In [5]:
URLs = []
for page in cached_pages:
    with open(page) as fp:
        html_content = BeautifulSoup(fp, 'html.parser')  
    content = html_content.findAll('a')
    content = [i.get('href') for i in content if (not (i.get('href') is None))]
    for link in content:
        URLs.append([name,link])
    print(content) 

['https://www.jgballard.ca/non_fiction/jgb_fictions.html', 'https://hakaimagazine.com/features/greenlands-melting-ice-sheet-brings-an-unexpected-flow-of-wealth-potential/?utm_source=substack&utm_medium=email', 'https://www.theverge.com/2022/10/25/23422359/shutterstock-ai-generated-art-openai-dall-e-partnership-contributors-fund-reimbursement', 'https://www.axa.com/en/magazine/2022-future-risks-report?utm_source=substack&utm_medium=email', 'https://www.rentcafe.com/blog/rental-market/market-snapshots/adaptive-reuse-apartments/?utm_source=newsletter&utm_medium=email&utm_campaign=newsletter_axioswhatsnext&stream=science', 'https://wordcraft-writers-workshop.appspot.com/learn?utm_source=substack&utm_medium=email', 'https://wordcraft-writers-workshop.appspot.com/stories/ken-liu', 'https://seenapse.it/?utm_source=substack&utm_medium=email', 'https://dl.acm.org/doi/pdf/10.1145/3564017', 'https://futurescanvas.com/en', 'https://modemworks.com/research/patented-futures/?utm_source=substack&utm_

['https://www.cbr.com/bruce-willis-sells-deepfake-likeness/', 'https://www.autocar.co.uk/car-news/consumer/cost-public-ev-charging-rises-more-42-four-months', 'https://www.nytimes.com/2022/09/26/business/factory-jobs-workers-rebound.html', 'https://www.bloomberg.com/news/articles/2021-10-22/how-paris-will-become-100-cyclable', 'https://2012.osmz.ru/#15/24.8472/89.3763', 'https://www.fao.org/3/cb9910en/cb9910en.pdf?utm_source=substack&utm_medium=email', 'https://techpandemic.theglassroom.org/#', 'https://www.bloomberg.com/news/articles/2022-09-30/palantir-had-plan-to-crack-uk-health-system-buying-our-way-in', 'https://www.vice.com/en/article/y3pnkw/us-military-bought-mass-monitoring-augury-team-cymru-browsing-email-data', 'https://bostonreview.net/articles/the-banality-of-surveillance/', 'https://www.economist.com/business/2022/05/30/corporate-espionage-is-entering-a-new-era', 'https://www.peacetalksinternational.com/post/conflict', 'https://rishad.substack.com/p/a-new-esg', 'https://ww

In [6]:
import hashlib

df = pd.DataFrame(URLs)
df.columns = ["page","url"]
df = df.drop_duplicates(subset=["url"])
df["url"] = df.url.apply(lambda x: str(x).encode('utf-8'))

df["hash"] = df.url.apply(lambda x: hashlib.md5(str(x).encode('utf-8')).hexdigest())
df.to_parquet("urls.parquet.gzip",  engine='pyarrow', compression='gzip')
df

,page,url,hash
0,20230312,b'https://www.jgballard.ca/non_fiction/jgb_fic...,8ebb431ce8d949d5a1ecb52bbd42a69f
1,20230312,b'https://hakaimagazine.com/features/greenland...,6c6a5e8d50e3045323cf202ba5b17ea9
2,20230312,b'https://www.theverge.com/2022/10/25/23422359...,99ffe526026a0494fc25d44aba64bd70
3,20230312,b'https://www.axa.com/en/magazine/2022-future-...,792ea5817e7c3331760ac18e3090869b
4,20230312,b'https://www.rentcafe.com/blog/rental-market/...,8bff520dc945204951b32a6c4b10571e
...,...,...,...
968,20230312,b'https://knowablemagazine.org/article/health-...,c47cb1fe2b4ae786edcfe793f7c528c2
969,20230312,b'https://blog.seekwell.io/gpt3',492f4356567de26e0afe1f008454e899
971,20230312,"b'https://substackcdn.com/image/fetch/f_auto,q...",07a77cb65e9b99211c078e05f90f6eca
972,20230312,b'https://en.wikipedia.org/wiki/Valeriepieris_...,6ab95ef7d3a59b67b7851af561ed903b
